In [1]:
from owlready2 import *
import datetime
import re
import logging
import editdistance
from pprint import pprint
from nltk.corpus import wordnet as wn

In [3]:
# ontology setup
onto = get_ontology("resources/ISTAT_RDF_XML.OWL")
onto.load()
sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /home/rana/Envs/ontoenv/lib/python3.5/site-packages/owlready2/hermit:/home/rana/Envs/ontoenv/lib/python3.5/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpa0ivnxnt
* Owlready2 * HermiT took 1.6774778366088867 seconds
* Owlready * Reparenting ISTAT_RDF_XML.OWL.coniuge_di: {ISTAT_RDF_XML.OWL.relazione_parentale, owl.ObjectProperty} => {ISTAT_RDF_XML.OWL.relazione_parentale}
* Owlready * Reparenting ISTAT_RDF_XML.OWL.nipote_di: {ISTAT_RDF_XML.OWL.relazione_parentale, owl.ObjectProperty} => {ISTAT_RDF_XML.OWL.relazione_parentale}
* Owlready * Reparenting ISTAT_RDF_XML.OWL.fratello_di: {ISTAT_RDF_XML.OWL.relazione_parentale, owl.ObjectProperty} => {ISTAT_RDF_XML.OWL.relazione_parentale}
* Owlready * Reparenting ISTAT_RDF_XML.OWL.nonno_di: {ISTAT_RDF_XML.OWL.relazione_parentale, owl.ObjectProperty} => {ISTAT_RDF_XML.OWL.relazione_parentale}


In [13]:
# logger setup
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [14]:
def preprocess_string(s, lower = True, stem = True, remove_stopwords = True, remove_punctuation = True):
    
    # lower every char
    # entity recognition also uses upper chars
    if lower: s = s.lower()
    
    # replace accents
    accent_chars = {
        "è":"e",
        "é":"e",
        "à":"a",
        "ò":"o",
        "ó":"o",
        "ù":"u",
        "ì":"i",
    }
    
    for char in accent_chars: s.replace(char, accent_chars[char])
        
    # tokenize and remove punctuation
    from nltk.tokenize import RegexpTokenizer
    
    tokenizer = RegexpTokenizer(r"\w+")
    s = tokenizer.tokenize(s)
        
    # stem the words
    if stem: 
        
        from nltk.stem.snowball import ItalianStemmer
        
        stemmer = ItalianStemmer()
        s = [stemmer.stem(word) for word in s]
    
    # remove stopwords (italian)
    if remove_stopwords: 
        
        from stop_words import get_stop_words
        
        s = [word for word in s if word not in get_stop_words('it')]
        
    return " ".join(s)

In [15]:
#1

domande = []

for c in onto.classes():
    
    # i verbi sono preceduti da "che" nelle object properties?
    
    # controlla subclasses
    for c2 in c.ancestors():
        if c2.name != "Thing" and c2.name != c.name:
            question_string = c2.name.replace("_", " ") + " che è " + c.name.replace("_", " ")
            domande.append(question_string)
            
            if c2.name in c.name:
                question_string = c2.name.replace("_", " ") + " che è " + c.name.replace(c2.name, "").replace("_", " ")
                question_string = re.sub("\s{2,}", " ", question_string)
                domande.append(question_string)
    
    # controlla object properties
    for p in onto.object_properties():
        
        if c in p.domain:
            
            question_string = ""
            question_string += c.name.replace("_", " ") + " "
            question_string += p.name.replace("_", " ") + " "
            
            for r in p.range:
                question_string += "[" + r.name + "]"
            
            domande.append(question_string)
            
    # controlla data properties
    for p in onto.data_properties():
        
        if c in p.domain:
            
            question_string = ""
            question_string += c.name.replace("_", " ") + " con "
            question_string += p.name.replace("_", " ") + " "
            
            for r in p.range:
                
                if r == str:
                    question_string += "[STR]"
                elif r == int:
                    question_string += "[INT]"
                elif r == datetime.datetime:
                    question_string += "[DATE]"
                    
            domande.append(question_string)
            
logger.debug("created " + str(len(domande)) + " questions")
for d in domande:
    print(d)

Relazione parentale che è Altro legame con lo intestatario non altrove specificato
Persona in vita che è Apolide
Persona che è Apolide
Entita statistico geografica che è Area censimento
Area censimento si tova in localita [Localita]
Nucleo abitato che è Capoluogo
Localita che è Capoluogo
Entita statistico geografica che è Capoluogo
Centro abitato che è Capoluogo
Case sparse e in comune [Comune]
Nucleo abitato che è Centro abitato
Entita statistico geografica che è Centro abitato
Localita che è Centro abitato
Nucleo abitato che è Centro post calamita
Entita statistico geografica che è Centro post calamita
Localita che è Centro post calamita
Nucleo abitato che è Centro temporaneo
Entita statistico geografica che è Centro temporaneo
Localita che è Centro temporaneo
Persona che è Cittadino italiano
Persona in vita che è Cittadino italiano
Persona che è Cittadino straniero
Persona in vita che è Cittadino straniero
Relazione parentale che è Cognato a dello intestatario
Entita amministrativa 

In [16]:
def find_most_similar(l1, s):
    best = float('inf')
    best_match = ""
    
    for l in l1:
        ed = editdistance.eval(l, s)
        if ed < best:
            best = ed
            best_match = l
            
    logger.debug("best_match: " + best_match)
    logger.debug("levenshtein distance: " + str(best))
            
    return best_match

In [17]:
from lib.treetagger import TreeTagger

tt = TreeTagger(language='italian')
tt.tag('lupa')

[['lupa', 'NOM', 'lupa']]

In [18]:
find_most_similar(domande, "persona che è cittadino")
preprocess_string("persona che è cittadino")

'person cittadin'